### Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle

### Feature Engineering (та же функция, что и в train)

In [2]:
def feat_engin(df):
    data = []
    for i in range(len(df)):
        values = np.log1p(np.abs(df['values'][i])) * np.sign(df['values'][i])
        dates = pd.to_datetime(df['dates'][i])
        
        features = {
            'id': df['id'][i],
            'mean': np.mean(values),
            'std': np.std(values),
            'min': np.min(values),
            'max': np.max(values),
            'median': np.median(values),
            'q25': np.quantile(values, 0.25),
            'q75': np.quantile(values, 0.75),
            'range': np.max(values) - np.min(values),
            'trend': np.polyfit(range(len(values)), values, 1)[0],  
            'time_range': (dates.max() - dates.min()).days,
            'autocorr': pd.Series(values).autocorr(lag=1),
        }
        data.append(features)
    return pd.DataFrame(data)

### Загрузка тестовых данных и примение функции

In [3]:
test = pd.read_parquet('test.parquet')
test_f = feat_engin(test)

### Загрузка обученной модели

In [4]:
with open('xgb_model.pkl', 'rb') as f:
    model = pickle.load(f)

### Предсказание

In [5]:
pred = model.predict_proba(test_f.drop('id', axis=1))[:, 1]

D:\Anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:39:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


### Сохранение результатов

In [6]:
submission = pd.DataFrame({'id': test_f['id'], 'score': pred})
submission.to_csv('submission.csv', index=False)